In [1]:
import swiftest 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.colors as mcolors
from collections import namedtuple
plt.switch_backend('agg')

In [28]:
#!/usr/bin/env python3

"""
 Copyright 2023 - David Minton, Carlisle Wishard, Jennifer Pouplin, Jake Elliott, & Dana Singh
 This file is part of Swiftest.
 Swiftest is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License 
 as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.
 Swiftest is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
 of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.
 You should have received a copy of the GNU General Public License along with Swiftest. 
 If not, see: https://www.gnu.org/licenses. 
"""

"""
Creates a movie from a set of Swiftest output files. All simulation 
outputs are stored in the /simdata subdirectory.

Input
------
param.in    : ASCII Swiftest parameter input file.
data.nc     : A NetCDF file containing the simulation output.

Output
------
Chambers2013-aescatter.mp4  : A .mp4 file plotting eccentricity vs semimajor axis. 
OR 
Chambers2013-aiscatter.mp4  : A .mp4 file plotting inclination vs semimajor axis. 
"""
%env HDF5_USE_FILE_LOCKING=FALSE

import swiftest 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.colors as mcolors
from collections import namedtuple
plt.switch_backend('agg')

titletext = 'haumea'
valid_plot_styles = ["aescatter", "aiscatter"]#, "arotscatter"]
xlim={"aescatter" : (0.0, 5.0),
      "aiscatter" : (0.0, 5.0), 
      "arotscatter" : (0.0, 2.5)}
ylim={"aescatter" : (0.0, 1.0),
      "aiscatter" : (0.0, 40.0),
      "arotscatter" : (1.0, 10000.0)}
xlabel={"aescatter": r"Semimajor axis ($R_{Haumea}$)",
        "aiscatter": r"Semimajor axis ($R_{Haumea}$)",
        "arotscatter": "Semimajor axis (AU)"}
ylabel={"aescatter": "Eccentricity",
        "aiscatter": "Inclination (deg)",
        "arotscatter": "Rotation period (h)"}

cb_ax_long = 1.63 # longest axis of the central body

YR2HR = 365.25 * 24
ROT2PERIOD = YR2HR * 360.0

framejump = 1
origin_types = ["Initial conditions", "Merger", "Disruption", "Supercatastrophic", "Hit and run fragmentation"]


class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, ds, param):

        self.radscale = 2e6
        nframes = int(ds['time'].size / framejump)
        ds['rot_mag'] = swiftest.tool.magnitude(ds,"rot")
        ds['rot_mag'] = ROT2PERIOD / ds['rot_mag']
        self.Rcb = ds['radius'].sel(name="Centaur").isel(time=0).values[()]
        self.ds = ds
        self.param = param
        colors = ["k", "xkcd:faded blue", "xkcd:marigold", "xkcd:shocking pink", "xkcd:baby poop green"]
        self.clist = dict(zip(origin_types,colors))

        # Setup the figure and axes...
        fig = plt.figure(figsize=(8,4.5), dpi=300)
        plt.tight_layout(pad=0)
        # set up the figure
        self.ax = plt.Axes(fig, [0.1, 0.15, 0.8, 0.75])
        fig.add_axes(self.ax)

        self.make_artists()
        
        self.ani = animation.FuncAnimation(fig, func=self.update, interval=1, frames=nframes, init_func=self.init_plot, blit=True)
        self.ani.save(animation_file, fps=60, dpi=300, extra_args=['-vcodec', 'libx264'])
        print(f'Finished writing {animation_file}')
        
    def make_artists(self):
        scatter_names = [f"s{i}" for i,k in enumerate(origin_types)]
        self.scatter_artist_names = dict(zip(origin_types,scatter_names))
        
        animated_elements = [self.ax.text(0.50, 1.05, "", bbox={'facecolor': 'w', 'alpha': 0.5, 'pad': 5}, transform=self.ax.transAxes,ha="center", animated=True)]
        element_names = ["title"]
        for key, value in self.clist.items():
            animated_elements.append(self.ax.scatter([], [], marker='o', s=[], c=value, alpha=0.75, label=key, animated=True))
            element_names.append(self.scatter_artist_names[key])
        
        Artists = namedtuple("Artists",tuple(element_names))
        self.artists = Artists(*animated_elements)
        return 

    def init_plot(self):
        self.ax.set_xlim(xlim[plot_style])
        self.ax.set_ylim(ylim[plot_style])
        
        # set up the figure
        self.ax.margins(x=10, y=1)
        self.ax.set_xlabel(xlabel[plot_style], fontsize='16', labelpad=1)
        self.ax.set_ylabel(ylabel[plot_style], fontsize='16', labelpad=1) 
        
        leg = plt.legend(loc="upper left", scatterpoints=1, fontsize=10)
        for i,l in enumerate(leg.legendHandles):
            leg.legendHandles[i]._sizes = [20]
        
        if plot_style == "arotscatter":
            self.ax.set_yscale('log')
        
        # plot longest axis
        self.ax.plot([cb_ax_long, cb_ax_long], ylim[plot_style], '-', color = 'orange')
        self.ax.text(cb_ax_long - 0.1, ylim[plot_style][-1] / 5, s = r'Longest Haumea Axis', color = 'orange', rotation = 'vertical')

        return self.artists

    def get_data(self, frame=0):
        d = self.ds.isel(time = frame)
        n=len(d['name'])
        d = d.isel(name=range(1,n))
        d['radmarker'] = (d['radius'] / self.Rcb) * self.radscale

        t = d['time'].values
        npl = d['npl'].values
        radmarker = d['radmarker'].values
        origin = d['origin_type'].values

        # print(radmarker)

        # Check for test particles
        # radmarker = np.where(np.isnan(radmarker), 2.0, radmarker)

        if plot_style == "aescatter":
            pl = np.c_[d['a'].values,d['e'].values]
        elif plot_style == "aiscatter":
            pl = np.c_[d['a'].values,d['inc'].values]
        elif plot_style == "arotscatter":
            pl = np.c_[d['a'].values,d['rot_mag'].values]

        return t, npl, pl, radmarker, origin

    def update(self,frame):
        """Update the scatter plot."""
        t, npl, pl, radmarker, origin = self.get_data(framejump * frame)

        self.artists.title.set_text(f"{titletext} - Time = ${t / 365.25:6.3f}$ years with ${npl:4.0f}$ particles")

        for key,name in self.scatter_artist_names.items():
            idx = origin == key
            if any(idx) and any(~np.isnan(radmarker[idx])):
                scatter = self.artists._asdict()[name]
                scatter.set_sizes(radmarker[idx])
                scatter.set_offsets(pl[idx,:])
                scatter.set_facecolor(self.clist[key])

        return self.artists

sim = swiftest.Simulation(simdir = "", read_data=True, read_collisions=False, dask=True)
for plot_style in valid_plot_styles:
    animation_file = titletext+f"-{plot_style}.mp4"
    print('Making animation')
    anim = AnimatedScatter(sim.data,sim.param)
    print('Animation finished')


env: HDF5_USE_FILE_LOCKING=FALSE
Reading Swiftest file /scratch/bell/anand43/swiftest/haumea/param.in

Creating Dataset from NetCDF file
Successfully converted 1 output frames.

Creating Dataset from NetCDF file
Successfully converted 16 output frames.
Swiftest simulation data stored as xarray DataSet .data
Reading initial conditions file as .init_cond
Finished reading Swiftest dataset files.
Making animation
Finished writing haumea-aescatter.mp4
Animation finished
Making animation
Finished writing haumea-aiscatter.mp4
Animation finished


In [14]:
# Plot inital a vs e to see initial conditions
sim = swiftest.Simulation(simdir = "", read_param=True, read_collisions=False)#, dask=True)

d = sim.data.isel(time = 0)
n=len(d['name'])
d = d.isel(name=range(1,n))

a = d['a'].values
e = d['e'].values
inc = d['inc'].values

fig, ax = plt.subplots(figsize=(8,4.5), dpi=300)

print(f"number of bodies = {n}")
print(a)
# print(e)

plt.scatter(a, e)
plt.xlabel('a')
plt.ylabel('e')
plt.savefig("a_vs_e_init.png")

plt.clf()
plt.scatter(a, inc)
plt.xlabel('a')
plt.ylabel('inc')
plt.savefig("a_vs_i_init.png")



Reading Swiftest file /scratch/bell/anand43/swiftest/haumea/param.in

Creating Dataset from NetCDF file
Successfully converted 1 output frames.

Creating Dataset from NetCDF file
Successfully converted 16 output frames.
Swiftest simulation data stored as xarray DataSet .data
Reading initial conditions file as .init_cond
Finished reading Swiftest dataset files.
number of bodies = 1001
[2.86707    2.26437611 3.6405438  3.93954489 2.94934881 3.08719425
 2.95619724 1.9315956  3.35814933 2.77744476 2.20363536 1.80574104
 2.71574397 3.49343986 2.03069888 3.5347133  2.46825806 2.71661343
 3.93758726 2.97675649 2.92302706 3.14238838 2.01017286 3.05107946
 2.9221084  3.83691596 2.32825779 2.63754725 3.33224001 2.45594668
 2.16867971 3.54655207 3.26450524 2.49896467 3.51590433 2.76106804
 1.9547691  2.74151039 3.94052577 2.13224769 3.36147178 2.53802787
 3.94673846 2.55988291 2.61847283 3.7620777  2.74105637 2.26478436
 1.81292722 2.76747095 2.29839042 3.80990784 3.62667666 3.71100367
 3.1725749

In [19]:
sim.data.isel(time=0)

<xarray.DataArray 'inc' (name: 1001)>
array([        nan, 10.17878648,  9.06981385, ...,  7.7303241 ,
        6.73890149,  9.62794807])
Coordinates:
    time     float64 0.0
  * name     (name) <U32 'Centaur' '100' '101' '102' ... '1097' '1098' '1099'
Attributes:
    _FillValue:  nan